In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px # ploty

from collections import Counter

In [ ]:
base_path = '/content/drive/MyDrive/이어드림스쿨/Project/Final_project/2021 농산물 예측대회 AI 경진대회/2021 농산물 가격예측 AI 경진대회/프로젝트 data/기상데이터/'

In [ ]:
df = pd.read_csv(base_path + '2016년 전국.csv', encoding = 'cp949')

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 825324 entries, 0 to 825323
Data columns (total 10 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   지점        825324 non-null  int64  
 1   지점명       825324 non-null  object 
 2   일시        825324 non-null  object 
 3   기온(°C)    825271 non-null  float64
 4   강수량(mm)   78782 non-null   float64
 5   풍속(m/s)   824999 non-null  float64
 6   습도(%)     825185 non-null  float64
 7   일조(hr)    450533 non-null  float64
 8   적설(cm)    9203 non-null    float64
 9   지면온도(°C)  824985 non-null  float64
dtypes: float64(7), int64(1), object(2)
memory usage: 63.0+ MB


In [ ]:
# 결측치는 다 0으로 처리!

display(df.isna().sum())

df1 = df.copy()

df1 = df1.fillna(0)
df1.isna().sum()

지점               0
지점명              0
일시               0
기온(°C)          53
강수량(mm)     746542
풍속(m/s)        325
습도(%)          139
일조(hr)      374791
적설(cm)      816121
지면온도(°C)       339
dtype: int64

지점          0
지점명         0
일시          0
기온(°C)      0
강수량(mm)     0
풍속(m/s)     0
습도(%)       0
일조(hr)      0
적설(cm)      0
지면온도(°C)    0
dtype: int64

## 2016년으로 진행

- 기온, 강수량, 풍속, 습도, 일조, 적설, 지면온도<br><br>

- 기온, 지면온도 - > 최저, 평균, 최고
- 풍속, 습도 -> 평균 
- 강수량, 적설량 -> 합계
- 일조량 -> 평균, 합계


In [ ]:
# 날짜별로 자른 columns 추가

df1["일시"] = pd.to_datetime(df1["일시"])
df1['일시(날짜)'] = df1["일시"].dt.date

In [ ]:
df1.head()

,지점,지점명,일시,기온(°C),강수량(mm),풍속(m/s),습도(%),일조(hr),적설(cm),지면온도(°C),일시(날짜)
0,90,속초,2016-01-01 01:00:00,0.1,0.0,1.8,49.0,0.0,0.0,-1.4,2016-01-01
1,90,속초,2016-01-01 02:00:00,0.0,0.0,1.9,53.0,0.0,0.0,-1.4,2016-01-01
2,90,속초,2016-01-01 03:00:00,-0.5,0.0,1.1,51.0,0.0,0.0,-1.7,2016-01-01
3,90,속초,2016-01-01 04:00:00,-0.3,0.0,1.0,55.0,0.0,0.0,-1.9,2016-01-01
4,90,속초,2016-01-01 05:00:00,2.5,0.0,2.6,45.0,0.0,0.0,-1.4,2016-01-01


In [ ]:
# 온도

pd.options.display.float_format = '{:.2f}'.format
df2 = df1.groupby(['지점명', '일시(날짜)'])['기온(°C)'].aggregate([np.min, np.mean, np.max])
df2.columns = ['온도(최저)', '온도(평균)', '온도(최고)',]
df2

온도(최저)  온도(평균)  온도(최고)
지점명 일시(날짜)                            
강릉  2016-01-01    2.30    4.90    8.70
    2016-01-02    5.50    8.67   11.00
    2016-01-03    5.50    9.46   13.70
    2016-01-04    4.10    7.62   11.60
    2016-01-05    0.60    3.23    7.20
...                ...     ...     ...
흑산도 2016-12-27    1.30    3.23    5.90
    2016-12-28    1.60    2.72    5.10
    2016-12-29    1.60    2.99    4.10
    2016-12-30    1.60    2.53    4.50
    2016-12-31    3.50    3.50    3.50

[34496 rows x 3 columns]

In [ ]:
df3 = df1.groupby(['지점명', '일시(날짜)'])['지면온도(°C)'].aggregate([np.min, np.mean, np.max])
df3.columns = ['지면온도(최저)', '지면온도(평균)', '지면온도(최고)',]
df3

지면온도(최저)  지면온도(평균)  지면온도(최고)
지점명 일시(날짜)                                  
강릉  2016-01-01      1.00      5.54     13.80
    2016-01-02      5.00      8.96     15.00
    2016-01-03      1.90      9.85     19.80
    2016-01-04      3.20      8.24     19.60
    2016-01-05     -0.40      4.10     16.00
...                  ...       ...       ...
흑산도 2016-12-27      4.40      5.79      6.90
    2016-12-28      3.80      5.59      9.10
    2016-12-29      4.30      5.13      6.60
    2016-12-30      3.60      5.34      8.70
    2016-12-31      4.40      4.40      4.40

[34496 rows x 3 columns]

In [ ]:
df4 = df1.groupby(['지점명', '일시(날짜)'])[['풍속(m/s)', '습도(%)']].aggregate([np.mean])
df4.columns = ['풍속(m/s)(평균)', '습도(%)(평균)']
df4

풍속(m/s)(평균)  습도(%)(평균)
지점명 일시(날짜)                            
강릉  2016-01-01         4.95      32.30
    2016-01-02         4.77      43.54
    2016-01-03         2.61      53.33
    2016-01-04         3.38      37.46
    2016-01-05         3.79      16.79
...                     ...        ...
흑산도 2016-12-27        13.65      79.42
    2016-12-28         6.99      73.42
    2016-12-29         9.16      71.00
    2016-12-30         4.22      78.38
    2016-12-31         0.90      73.00

[34496 rows x 2 columns]

In [ ]:
df5 = df1.groupby(['지점명', '일시(날짜)'])[['강수량(mm)', '적설(cm)']].aggregate([np.sum])
df5.columns = ['강수량(mm)(합계)', '적설량(cm)(합계)']
df5

강수량(mm)(합계)  적설량(cm)(합계)
지점명 일시(날짜)                              
강릉  2016-01-01         0.00         0.00
    2016-01-02         0.00         0.00
    2016-01-03         0.00         0.00
    2016-01-04         0.00         0.00
    2016-01-05         0.00         0.00
...                     ...          ...
흑산도 2016-12-27         0.00         0.00
    2016-12-28         0.00         0.00
    2016-12-29         0.00         0.00
    2016-12-30         0.00         0.00
    2016-12-31         0.00         0.00

[34496 rows x 2 columns]

In [ ]:
df6 = df1.groupby(['지점명', '일시(날짜)'])['일조(hr)'].aggregate([np.sum, np.mean])
df6.columns = ['일조량(합계)', '일조량(평균)']
df6

일조량(합계)  일조량(평균)
지점명 일시(날짜)                      
강릉  2016-01-01     8.80     0.38
    2016-01-02     8.70     0.36
    2016-01-03     7.20     0.30
    2016-01-04     8.40     0.35
    2016-01-05     7.60     0.32
...                 ...      ...
흑산도 2016-12-27     1.90     0.08
    2016-12-28     6.20     0.26
    2016-12-29     1.80     0.07
    2016-12-30     6.00     0.25
    2016-12-31     0.00     0.00

[34496 rows x 2 columns]

In [ ]:
# dataframe df2,3,4,5,6 합치기

In [ ]:
result = pd.concat([df2,df3, df4, df5, df6], axis = 1)
type(result)

pandas.core.frame.DataFrame

In [ ]:
result = result.reset_index()

In [ ]:
result.head()

,index,지점명,일시(날짜),온도(최저),온도(평균),온도(최고),지면온도(최저),지면온도(평균),지면온도(최고),풍속(m/s)(평균),습도(%)(평균),강수량(mm)(합계),적설량(cm)(합계),일조량(합계),일조량(평균)
0,0,강릉,2016-01-01,2.30,4.90,8.70,1.00,5.54,13.80,4.95,32.30,0.00,0.00,8.80,0.38
1,1,강릉,2016-01-02,5.50,8.67,11.00,5.00,8.96,15.00,4.77,43.54,0.00,0.00,8.70,0.36
2,2,강릉,2016-01-03,5.50,9.46,13.70,1.90,9.85,19.80,2.61,53.33,0.00,0.00,7.20,0.30
3,3,강릉,2016-01-04,4.10,7.62,11.60,3.20,8.24,19.60,3.38,37.46,0.00,0.00,8.40,0.35
4,4,강릉,2016-01-05,0.60,3.23,7.20,-0.40,4.10,16.00,3.79,16.79,0.00,0.00,7.60,0.32


# 함수화



In [ ]:
def pre(df1) :

    df1 = df1.fillna(0)
 
    df1["일시"] = pd.to_datetime(df1["일시"])
    df1['일시(날짜)'] = df1["일시"].dt.date

    pd.options.display.float_format = '{:.2f}'.format
    df2 = df1.groupby(['지점명', '일시(날짜)'])['기온(°C)'].aggregate([np.min, np.mean, np.max])
    df2.columns = ['온도(최저)', '온도(평균)', '온도(최고)',]
    
    df3 = df1.groupby(['지점명', '일시(날짜)'])['지면온도(°C)'].aggregate([np.min, np.mean, np.max])
    df3.columns = ['지면온도(최저)', '지면온도(평균)', '지면온도(최고)',]

    df4 = df1.groupby(['지점명', '일시(날짜)'])[['풍속(m/s)', '습도(%)']].aggregate([np.mean])
    df4.columns = ['풍속(m/s)(평균)', '습도(%)(평균)']
    
    df5 = df1.groupby(['지점명', '일시(날짜)'])[['강수량(mm)', '적설(cm)']].aggregate([np.sum])
    df5.columns = ['강수량(mm)(합계)', '적설량(cm)(합계)']

    df6 = df1.groupby(['지점명', '일시(날짜)'])['일조(hr)'].aggregate([np.sum, np.mean])
    df6.columns = ['일조량(합계)', '일조량(평균)']

    # dataframe 합치기
    result = pd.concat([df2,df3, df4, df5, df6], axis = 1)
    result = result.reset_index()

    return result


In [ ]:
df_2016 = pd.read_csv(base_path + '2016년 전국.csv', encoding = 'cp949')
df_2017 = pd.read_csv(base_path + '2017년 전국.csv', encoding = 'cp949')
df_2018 = pd.read_csv(base_path + '2018년 전국.csv', encoding = 'cp949')
df_2019 = pd.read_csv(base_path + '2019년 전국.csv', encoding = 'cp949')
df_2020 = pd.read_csv(base_path + '2020년 9월까지 전국.csv', encoding = 'cp949')

In [ ]:
result = pre(df_2016)
result2 = pre(df_2017)
result3 = pre(df_2018)
result4 = pre(df_2019)
result5 = pre(df_2020)

In [ ]:
display(result.head(2))
print(result.shape)
display(result2.head(2))
print(result2.shape)
display(result3.head(2))
print(result3.shape)
display(result4.head(2))
print(result4.shape)
display(result5.head(2))
print(result5.shape)

,지점명,일시(날짜),온도(최저),온도(평균),온도(최고),지면온도(최저),지면온도(평균),지면온도(최고),풍속(m/s)(평균),습도(%)(평균),강수량(mm)(합계),적설량(cm)(합계),일조량(합계),일조량(평균)
0,강릉,2016-01-01,2.30,4.90,8.70,1.00,5.54,13.80,4.95,32.30,0.00,0.00,8.80,0.38
1,강릉,2016-01-02,5.50,8.67,11.00,5.00,8.96,15.00,4.77,43.54,0.00,0.00,8.70,0.36


(34496, 14)


,지점명,일시(날짜),온도(최저),온도(평균),온도(최고),지면온도(최저),지면온도(평균),지면온도(최고),풍속(m/s)(평균),습도(%)(평균),강수량(mm)(합계),적설량(cm)(합계),일조량(합계),일조량(평균)
0,강릉,2017-01-01,4.20,6.44,8.70,-0.50,4.94,15.20,4.50,40.00,0.00,0.00,7.60,0.33
1,강릉,2017-01-02,6.60,8.54,11.60,0.80,6.80,18.80,4.00,44.58,0.00,0.00,8.80,0.37


(34669, 14)


,지점명,일시(날짜),온도(최저),온도(평균),온도(최고),지면온도(최저),지면온도(평균),지면온도(최고),풍속(m/s)(평균),습도(%)(평균),강수량(mm)(합계),적설량(cm)(합계),일조량(합계),일조량(평균)
0,강릉,2018-01-01,-1.90,1.29,5.20,-4.90,1.71,15.50,3.83,19.74,0.00,0.00,9.10,0.40
1,강릉,2018-01-02,-0.20,2.57,6.50,-4.90,1.71,14.20,2.92,22.92,0.00,0.00,8.70,0.36


(34674, 14)


,지점명,일시(날짜),온도(최저),온도(평균),온도(최고),지면온도(최저),지면온도(평균),지면온도(최고),풍속(m/s)(평균),습도(%)(평균),강수량(mm)(합계),적설량(cm)(합계),일조량(합계),일조량(평균)
0,강릉,2019-01-01,-2.50,-0.59,2.70,-3.90,-1.03,4.30,4.04,24.48,0.00,0.00,6.40,0.28
1,강릉,2019-01-02,-3.30,-0.13,6.10,-4.50,-0.79,8.70,3.80,22.12,0.00,0.00,9.10,0.38


(34740, 14)


,지점명,일시(날짜),온도(최저),온도(평균),온도(최고),지면온도(최저),지면온도(평균),지면온도(최고),풍속(m/s)(평균),습도(%)(평균),강수량(mm)(합계),적설량(cm)(합계),일조량(합계),일조량(평균)
0,강릉,2020-01-01,-2.30,1.60,6.00,-3.60,1.09,12.60,4.63,41.52,0.00,0.00,8.60,0.37
1,강릉,2020-01-02,0.70,3.37,7.60,-2.00,2.30,14.00,4.54,46.29,0.00,0.00,8.40,0.35


(26030, 14)


In [ ]:
# dataframe 합치기

final_result = pd.concat([result, result2, result3, result4, result5], axis = 0)
final_result

,지점명,일시(날짜),온도(최저),온도(평균),온도(최고),지면온도(최저),지면온도(평균),지면온도(최고),풍속(m/s)(평균),습도(%)(평균),강수량(mm)(합계),적설량(cm)(합계),일조량(합계),일조량(평균)
0,강릉,2016-01-01,2.30,4.90,8.70,1.00,5.54,13.80,4.95,32.30,0.00,0.00,8.80,0.38
1,강릉,2016-01-02,5.50,8.67,11.00,5.00,8.96,15.00,4.77,43.54,0.00,0.00,8.70,0.36
2,강릉,2016-01-03,5.50,9.46,13.70,1.90,9.85,19.80,2.61,53.33,0.00,0.00,7.20,0.30
3,강릉,2016-01-04,4.10,7.62,11.60,3.20,8.24,19.60,3.38,37.46,0.00,0.00,8.40,0.35
4,강릉,2016-01-05,0.60,3.23,7.20,-0.40,4.10,16.00,3.79,16.79,0.00,0.00,7.60,0.32
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26025,흑산도,2020-09-26,18.30,20.88,23.20,20.20,22.24,26.60,4.37,79.92,0.00,0.00,10.40,0.43
26026,흑산도,2020-09-27,18.80,21.25,25.80,18.90,21.95,27.20,1.87,79.46,0.00,0.00,10.80,0.45
26027,흑산도,2020-09-28,18.60,20.92,25.10,19.10,21.84,26.40,2.18,79.92,0.00,0.00,10.40,0.43
26028,흑산도,2020-09-29,18.20,20.51,24.50,18.70,21.73,26.70,3.83,80.75,0.00,0.00,7.20,0.30


In [ ]:
final_result.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 164609 entries, 0 to 26029
Data columns (total 14 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   지점명          164609 non-null  object 
 1   일시(날짜)       164609 non-null  object 
 2   온도(최저)       164609 non-null  float64
 3   온도(평균)       164609 non-null  float64
 4   온도(최고)       164609 non-null  float64
 5   지면온도(최저)     164609 non-null  float64
 6   지면온도(평균)     164609 non-null  float64
 7   지면온도(최고)     164609 non-null  float64
 8   풍속(m/s)(평균)  164609 non-null  float64
 9   습도(%)(평균)    164609 non-null  float64
 10  강수량(mm)(합계)  164609 non-null  float64
 11  적설량(cm)(합계)  164609 non-null  float64
 12  일조량(합계)      164609 non-null  float64
 13  일조량(평균)      164609 non-null  float64
dtypes: float64(12), object(2)
memory usage: 18.8+ MB


In [ ]:
final_result.지점명.nunique()

96

data 저장

In [ ]:
!pip install pyarrow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
final_result.to_parquet( '/content/drive/MyDrive/이어드림스쿨/Project/Final_project/2021 농산물 예측대회 AI 경진대회/2021 농산물 가격예측 AI 경진대회/프로젝트 data/기상데이터(전처리완료).parquet', engine="pyarrow", compression='gzip')